In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using DifferentialEquations
using MeshIO

[ Info: Precompiling VMRobotControl [7d923390-5bb2-4b12-9c06-5b46ef7ec375]
[ Info: Precompiling VMRobotControlMakieExt [eb905980-c17a-52fb-868a-56f829bc8445]
[ Info: Precompiling VMRobotControlDifferentialEquationsExt [8dceb1f5-daf4-58d1-992f-bbb034695ea0]


## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Hand Visualization

In [3]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)

# Display figure
display(fig)

GLMakie.Screen(...)

### Hand properties : Frames, Joints, Coordinates & Components

In [5]:
#println("FRAMES")
#display(frames(robot))
#println("JOINTS")
#display(joints(robot))
#show(IOContext(stdout, :limit => false),  MIME("text/plain"), joints(robot))
#println("COORDINATES")
#show(coordinates(robot))
show(IOContext(stdout, :limit => false),  MIME("text/plain"), coordinates(robot))
#println("COMPONENTS")
#display(components(robot))



Dict{String, VMRobotControl.CoordinateData} with 55 entries:
  "rh_lfdistal_mass_coord" => FramePoint{Float64, String}(rh_lfdistal, [0.0, 0.0, 0.012])
  "rh_rfdistal_mass_coord" => FramePoint{Float64, String}(rh_rfdistal, [0.0, 0.0, 0.012])
  "rh_thdistal_mass_coord" => FramePoint{Float64, String}(rh_thdistal, [0.0, 0.0, 0.01375])
  "rh_rftip_mass_coord" => FrameOrigin{String}(rh_rftip)
  "rh_palm_mass_coord" => FramePoint{Float64, String}(rh_palm, [0.0, 0.0, 0.035])
  "rh_rfknuckle_mass_coord" => FrameOrigin{String}(rh_rfknuckle)
  "rh_wrist_mass_coord" => FramePoint{Float64, String}(rh_wrist, [0.0, 0.0, 0.029])
  "rh_ffdistal_inertia_coord" => FrameAngularVelocity{String}("rh_ffdistal")
  "rh_thhub_mass_coord" => FrameOrigin{String}(rh_thhub)
  "rh_rfmiddle_mass_coord" => FramePoint{Float64, String}(rh_rfmiddle, [0.0, 0.0, 0.0125])
  "rh_mftip_mass_coord" => FrameOrigin{String}(rh_mftip)
  "rh_ffdistal_mass_coord" => FramePoint{Float64, String}(rh_ffdistal, [0.0, 0.0, 0.012])
  "rh_m

## Creating the Virtual Mechanism System

### Gravity Compensation and Joint Damping

Identify the actuated joints

In [6]:
number_of_actuated_joints = 0
actuated_joints = String[]

for (joint_name, joint) in joints(robot)
    if(typeof(joint).parameters[1] <: RevoluteData)
        number_of_actuated_joints += 1
        push!(actuated_joints, joint_name)
    end
end

println("Number of actuated joints :" , number_of_actuated_joints)
print("List of the actuated joints :")
display(actuated_joints)

Number of actuated joints :24
List of the actuated joints :

24-element Vector{String}:
 "rh_WRJ2"
 "rh_WRJ1"
 "rh_FFJ4"
 "rh_FFJ3"
 "rh_FFJ2"
 "rh_FFJ1"
 "rh_MFJ4"
 "rh_MFJ3"
 "rh_MFJ2"
 "rh_MFJ1"
 "rh_RFJ4"
 "rh_RFJ3"
 "rh_RFJ2"
 "rh_RFJ1"
 "rh_LFJ5"
 "rh_LFJ4"
 "rh_LFJ3"
 "rh_LFJ2"
 "rh_LFJ1"
 "rh_THJ5"
 "rh_THJ4"
 "rh_THJ3"
 "rh_THJ2"
 "rh_THJ1"

Add Gravity Compensation

In [3]:
add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

Add joint limiting springs and linear damper

In [4]:
joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Virtual Mechanism Initialization

Equivalent de "TCP position" : "rh_fftip_mass_coord" => FrameOrigin{String}(rh_fftip)

In [5]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)
#add_coordinate!(vms, FramePoint(".robot.$root", SVector(0.0, -0.1, 0.25));        id="Target position")



add_coordinate!(vms, FrameOrigin(".robot.rh_palm");        id="Target position")
add_coordinate!(vms, CoordDifference(".robot.rh_fftip_mass_coord", "Target position"); id="Position error");


In [6]:
K = SMatrix{3, 3}(100., 0., 0., 0., 100., 0., 0., 0., 100.)
add_component!(vms, LinearSpring(K, "Position error");           id="Linear Spring")
D = SMatrix{3, 3}(100., 0., 0., 0., 100.0, 0., 0., 0., 100.)
add_component!(vms, LinearDamper(D, "Position error");           id="Linear Damper")

"Linear Damper"

## Simulating the Robot

### Setting Up the Simulation

In [7]:
disturbance_func(t) = mod(t, 6) < 3 ? SVector(0., 0., 0.) : SVector(0., 0., 10.)

f_setup(cache) = get_compiled_coordID(cache, ".robot.rh_fftip_mass_coord")

function f_control(cache, t, args, extra)
    tcp_pos_coord_id = args
    F = disturbance_func(t)
    uᵣ, uᵥ = get_u(cache)
    z = configuration(cache, tcp_pos_coord_id)
    J = jacobian(cache, tcp_pos_coord_id)
    mul!(uᵣ, J', F)
    nothing
end

tspan = (0., 6.)
vms_compiled = compile(vms)
q = (zero_q(vms_compiled.robot), Float64[]) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), Float64[]) # Robot joint velocity, vm joint velocities
g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with impedance control."

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

[ Info: Simulating shadow robot with impedance control.
ODE   0%|█                                              |  ETA: N/A
ODE   2%|█                                              |  ETA: 0:02:31
ODE   4%|██                                             |  ETA: 0:01:43
ODE   6%|████                                           |  ETA: 0:01:24
ODE   9%|█████                                          |  ETA: 0:01:17
ODE  11%|██████                                         |  ETA: 0:01:11
ODE  13%|███████                                        |  ETA: 0:01:06
ODE  16%|████████                                       |  ETA: 0:01:02
ODE  18%|█████████                                      |  ETA: 0:01:00
ODE  20%|██████████                                     |  ETA: 0:00:57
ODE  22%|███████████                                    |  ETA: 0:00:55
ODE  25%|████████████                                   |  ETA: 0:00:52
ODE  27%|█████████████                                  |  ETA: 0:00:50
ODE  29%|███

### Visualizing the results

In [8]:
fig = Figure(size = (720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls, camera=:perspective, center=false)
cam.lookat[] = [0.25, 0.5, 0.1]
cam.eyeposition[] = [-0.4, -0.8, 0.5]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(robot)))
robotvisualize!(ls, plotting_kcache;)

tcp_pos_id = get_compiled_coordID(plotting_kcache[], "rh_fftip_mass_coord")
tcp_pos = map(plotting_kcache) do kcache
    Point3f(configuration(kcache, tcp_pos_id))
end
force = map(t -> 0.01 * Vec3f(disturbance_func(t)), plotting_t)
arrowsize = map(f -> 0.1*(f'*f)^(0.25), force)
arrows!(ls, map(p -> [p], tcp_pos), map(f -> [f], force); color = :red, arrowsize)

savepath = joinpath(module_path, "docs/src/assets/shadowhand_first_test.mp4")
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);